In [142]:
#Library to read input file
import pandas as pd
#For math operations
import numpy as np
# Used for splitting training data into validation and training set
from sklearn.model_selection import train_test_split

#Used for making TF-IDF vector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#used for computing precision_recall_fscore
from sklearn.metrics import precision_recall_fscore_support

#used for computing accuracy
from sklearn.metrics import accuracy_score

#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

#SVM classifier
from sklearn.svm import SVC

#Compute F1-score
from sklearn.metrics import f1_score

#Compute Precision 
from sklearn.metrics import precision_score

#Computes Recall
from sklearn.metrics import recall_score

# NLTK lib for word tokenization and other pre-processing
from nltk import word_tokenize
import nltk


from nltk.corpus import stopwords
from nltk import word_tokenize,WordNetLemmatizer


import re


In [143]:
#Reading trainingData file
data = pd.read_csv('training.csv',header=None, names=['text','character','gender'])

#Showing first 5 values of the training data file
data.head()

 
 

,text,character,gender
0,Someone had fun.,SEAN,male
1,"It's no problem, honestly. Go on, go and open ...",SHIRLEY,female
2,Last night was better than ever. What's all th...,MAX,male
3,Have you checked the answerphone? Any calls?,IAN,male
4,Oscar's asleep.,MAX,male


In [144]:
#PRE-PROCESSING
stoplist = stopwords.words('english')
#lemmatizes 
def lem_message(text):
    temp=[]
    text = text.split()
    wl=WordNetLemmatizer()
    for i in text:
        temp.append(wl.lemmatize(i,pos='v'))
    return " ".join(temp)

#this function removes stopwords
def remove_stopwords(text,stoplist):
    temp=[]
#     print(text)
#     print (len(text))
    text = text.split()
    for i in range (len(text)):
        if text[i] not in stoplist:
            temp.append(text[i])
    return " ".join(temp)

import string
x = string.punctuation
for i in range (data.shape[0]):
    data["text"].iloc[i] = re.sub('['+x+']' ,'',str(data["text"].iloc[i])) #thi removes special chars use this only :p for every sentence in train and test
    data["text"].iloc[i] = lem_message(str(data["text"].iloc[i]))
    data["text"].iloc[i] = remove_stopwords(data["text"].iloc[i],stoplist)
    


In [145]:
type(data["text"].iloc[0])

str

In [146]:
#pre-processing text/dialouges 
data['text']=data['text'].str.lower() # lower case
corpus=data['text'].values.astype('U') #went through the dat aand gave 

# Making tf-idf vector of Text data
vectorizer = TfidfVectorizer(ngram_range=(1,2)) #First word cant be bigram
#vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

#Vocabulory
featureNames=vectorizer.get_feature_names()

 

(10113, 39564)


In [147]:
# encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
 
labelencoder_Y = LabelEncoder()

# selecting gender column as class label. Ignoring character filed 
Y= labelencoder_Y.fit_transform(data.gender)

#splitting trining data, so that parameters can be fine tuned on 33% validation data
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [148]:
#This function does evaluation

def evaluate(y_test,y_pred,algoText='Random Forest (Baseline-1)'):
    print('------------------------------------------------------------------')
    print('Results of  ',algoText)

    print('------------------------------------------------------------------')


    print('Accuracy    ',accuracy_score(y_test, y_pred))

    print('Recall      ',recall_score(y_test, y_pred))
    
    print('Precision   ',precision_score(y_test,y_pred))

    print('F1-Score    ',f1_score(y_test,y_pred))
    
    

In [149]:
#applying random Forest algorithm

clfRF = RandomForestClassifier(n_estimators=100, min_samples_split=2)
clfRF.fit(X_train, y_train) 
y_pred=clfRF.predict(X_test)
evaluate(y_test,y_pred,'Random Forest (Baseline-1) on splitted_test')


------------------------------------------------------------------
Results of   Random Forest (Baseline-1) on splitted_test
------------------------------------------------------------------
Accuracy     0.5554224086279209
Recall       0.5509723040659988
Precision    0.5642727821363911
F1-Score     0.5575432319618366


In [150]:
#Apply linear SVM algorithm

clfSVM=SVC(gamma='auto',kernel='linear', C = 0.05)
clfSVM.fit(X_train, y_train) 
y_pred=clfSVM.predict(X_test)
print (y_pred)
evaluate(y_test,y_pred,'SVM (Baseline-2) on splitted_test')



[0 0 0 ... 0 0 0]
------------------------------------------------------------------
Results of   SVM (Baseline-2) on splitted_test
------------------------------------------------------------------
Accuracy     0.4919113241461953
Recall       0.012374779021803181
Precision    0.5121951219512195
F1-Score     0.024165707710011506


In [151]:
#Reading testing data by using Pandas Library

#TEST DATA WITHOUT POS TAGS----------------------------------
testData = pd.read_csv('test.csv',header=None, names=['character','text','gender'], dtype=str)

testData.head()

#PRE-PROCESSING------------------------------
stoplist = stopwords.words('english')
#lemmatizes 
def lem_message(text):
    temp=[]
    text = text.split()
    wl=WordNetLemmatizer()
    for i in text:
        temp.append(wl.lemmatize(i,pos='v'))
    return " ".join(temp)

#this function removes stopwords
def remove_stopwords(text,stoplist):
    temp=[]
#     print(text)
#     print (len(text))
    text = text.split()
    for i in range (len(text)):
        if text[i] not in stoplist:
            temp.append(text[i])
    return " ".join(temp)

import string
x = string.punctuation
for i in range (data.shape[0]):
    data["text"].iloc[i] = re.sub('['+x+']' ,'',str(data["text"].iloc[i])) #thi removes special chars use this only :p for every sentence in train and test
    data["text"].iloc[i] = lem_message(str(data["text"].iloc[i]))
    data["text"].iloc[i] = remove_stopwords(data["text"].iloc[i],stoplist)
    
#------------------------------------------------
    
#Textual Pre-processing
testData['text']=testData['text'].str.lower()
corpus=testData['text'].values.astype('U')



vectorizer = TfidfVectorizer(vocabulary=featureNames,ngram_range=(1,2))
tX = vectorizer.fit_transform(corpus)
labelencoder_Y = LabelEncoder()

# gender label is transformed 
y_Test= labelencoder_Y.fit_transform(testData.gender)

In [152]:
#Testing random forest model on test data
y_Pred=clfRF.predict(tX)
evaluate(y_Test,y_Pred,'Random Forest (Baseline-1) on Test Dataset') 

print(np.unique(y_Pred))

------------------------------------------------------------------
Results of   Random Forest (Baseline-1) on Test Dataset
------------------------------------------------------------------
Accuracy     0.4288256227758007
Recall       0.5903010033444817
Precision    0.4706666666666667
F1-Score     0.5237388724035609
[0 1]


In [153]:
#Testing SVM model on test data
y_Pred=clfSVM.predict(tX)
evaluate(y_Test,y_Pred,'SVM (Baseline-2) on Test Dataset')
# set(y_Pred)
# f1_score(y_Test,y_Pred)
# print(np.unique(y_Pred))

------------------------------------------------------------------
Results of   SVM (Baseline-2) on Test Dataset
------------------------------------------------------------------
Accuracy     0.4679715302491103
Recall       0.0
Precision    0.0
F1-Score     0.0


c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [154]:
#This function extracts POS tagging for each word of the sentence and add it with the sentence
def POSBasedFeatures(data):
    print(len(data))
    taggedData=[]
    for row in data:
        
        d=''
        #print('...')
        #print(row)
        tokens = word_tokenize(str(row))
        #nltk library to extract POS tagging
        tagged=nltk.pos_tag(tokens)
        
        #print(tagged)
        if tagged:
            for t in tagged:
                #print(t)
                d=d+ ' '+t[1] +' '+t[0]
#                 print (d)
            taggedData.append(d)
        else:
            d="NAN"+ ' '+"NAN" +' '+"NAN"
            taggedData.append(d)
        #print(tagged)
    print(len(taggedData))
    return taggedData
    
    
    

In [155]:
posData=POSBasedFeatures(data['text'])


10113
10113


In [156]:
#making tf-idf vector ngram=(1,2) means unigram and bigram tokens are used. 
# Rest of the code is same as the one as without applying POS
vectorizer = TfidfVectorizer(lowercase=False,ngram_range=(1,2))

#vectorizer = CountVectorizer()

X = vectorizer.fit_transform(posData)
print(X.shape)
# X1 = vectorizer.fit_transform(posData[2827:])
# for i in X:
#     print(i)
featureNames=vectorizer.get_feature_names()



(10113, 23735)


In [157]:
# encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
 
labelencoder_Y = LabelEncoder()

#Y = labelencoder_Y.fit_transform(data.gender)
Y= labelencoder_Y.fit_transform(data.gender)
#posdata split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [158]:
 
clfRF = RandomForestClassifier(n_estimators=100, min_samples_split=4)
clfRF.fit(X_train, y_train) 
y_pred=clfRF.predict(X_test)
evaluate(y_test,y_pred,'Random Forest (Baseline-1) on splitted_test')


------------------------------------------------------------------
Results of   Random Forest (Baseline-1) on splitted_test
------------------------------------------------------------------
Accuracy     0.5590173756740563
Recall       0.6128461991750147
Precision    0.5606469002695418
F1-Score     0.5855855855855856


In [159]:
clfSVM=SVC(gamma='auto',kernel='linear', C = 0.05)
clfSVM.fit(X_train, y_train) 
y_pred=clfSVM.predict(X_test)
evaluate(y_test,y_pred,'SVM (Baseline-2) on splitted_test')



------------------------------------------------------------------
Results of   SVM (Baseline-2) on splitted_test
------------------------------------------------------------------
Accuracy     0.5011983223487118
Recall       0.038892162639952856
Precision    0.66
F1-Score     0.07345575959933222


In [160]:
#Test data WITH POS TAGS-----------

posData=POSBasedFeatures(testData['text'])

vectorizer = TfidfVectorizer(vocabulary=featureNames,ngram_range=(1,2))
tX = vectorizer.fit_transform(posData)
labelencoder_Y = LabelEncoder()

#Y = labelencoder_Y.fit_transform(data.gender)
y_Test= labelencoder_Y.fit_transform(testData.gender)

1124
1124


In [161]:
y_Pred=clfRF.predict(tX)
evaluate(y_Test,y_Pred,'Random Forest (Baseline-1) on Test Dataset') 

print(np.unique(y_Pred))

------------------------------------------------------------------
Results of   Random Forest (Baseline-1) on Test Dataset
------------------------------------------------------------------
Accuracy     0.18505338078291814
Recall       0.0802675585284281
Precision    0.11594202898550725
F1-Score     0.09486166007905138
[0 1]


In [162]:
y_Pred=clfSVM.predict(tX)
evaluate(y_Test,y_Pred,'SVM (Baseline-2) on Test Dataset')
f1_score(y_Test,y_Pred)
# print(np.unique(y_Pred))

------------------------------------------------------------------
Results of   SVM (Baseline-2) on Test Dataset
------------------------------------------------------------------
Accuracy     0.4679715302491103
Recall       0.0
Precision    0.0
F1-Score     0.0


c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0